## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install xyz-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.8 MB/s eta 0:00:00


In [4]:
# Imports
import os
import sys
import numpy as np
import pandas as pd
import glob

import xyz_py as xyzp

matches = glob.glob("/content/drive/MyDrive/**/CDS_Group_Project", recursive=True)
if matches:
    BASE_DIR = matches[0]
else:
    raise FileNotFoundError("SharedProject folder not found in your Google Drive.")

data_dir = os.path.join(BASE_DIR, "Python_Notebooks/Group8_CDS_Data")

## Data exploration

In [6]:
# Define the raw data
label_df = pd.read_csv(f"{data_dir}/rates_ml.csv")
silanos_folder = f"{data_dir}/silanols_vicinal"

In [19]:
# Get a list of all files in the silanos folder
file_list = os.listdir(silanos_folder)

print(f"Filenames: \n{file_list[:4]} ...\n" )
print(f"There are {len(file_list)} files in the silanos_vicinal folder")

Filenames: 
['coords_1Kps_001_0001.xyz', 'coords_1Kps_002_0002.xyz', 'coords_1Kps_002_0103.xyz', 'coords_1Kps_003_0003.xyz'] ...

There are 388 files in the silanos_vicinal folder


In [14]:
# Let's explore one example of the .xyz data
test_coords = xyzp.load_xyz(f"{silanos_folder}/coords_1Kps_001_0001.xyz")

print(f"Atom names:\n{test_coords[0]}\n")
print(f"Full coordinates:\n{test_coords[1]}\n")
print(f"Coordinates of first atom:\n{test_coords[1][0]}\n")

Atom names:
['F', 'F', 'F', 'F', 'Si', 'Si', 'O', 'O', 'O', 'H', 'H']

Full coordinates:
[[-2.11975178  1.50069892 -0.36361168]
 [-2.37533734 -1.23103682 -0.71987053]
 [ 2.33552808 -1.32539002 -0.55981726]
 [ 1.99841315  1.25144317 -0.89070369]
 [-1.58748406  0.          0.        ]
 [ 1.58748406  0.          0.        ]
 [-0.01888556 -0.18871356 -0.28932679]
 [-1.82068638 -0.21080748  1.69100683]
 [ 1.79733515  0.21080748  1.69318366]
 [-2.71579025 -0.44376802  1.90780634]
 [ 1.2033109  -0.3244406   2.20618875]]

Coordinates of first atom:
[-2.11975178  1.50069892 -0.36361168]



In [24]:
# Make a list to iterate over and eventually add to dataframe
coord_list = [xyzp.load_xyz(os.path.join(silanos_folder, filename)) for filename in file_list]


In [25]:
# note, logs of the kinetic constants were used (nevermind not for the unsupervised learning part)
print(f"{coord_list[0][0]}\n")
print(f"{coord_list[0][1]}\n")
print(f"{coord_list[0][1][0]}")

['F', 'F', 'F', 'F', 'Si', 'Si', 'O', 'O', 'O', 'H', 'H']

[[-2.11975178  1.50069892 -0.36361168]
 [-2.37533734 -1.23103682 -0.71987053]
 [ 2.33552808 -1.32539002 -0.55981726]
 [ 1.99841315  1.25144317 -0.89070369]
 [-1.58748406  0.          0.        ]
 [ 1.58748406  0.          0.        ]
 [-0.01888556 -0.18871356 -0.28932679]
 [-1.82068638 -0.21080748  1.69100683]
 [ 1.79733515  0.21080748  1.69318366]
 [-2.71579025 -0.44376802  1.90780634]
 [ 1.2033109  -0.3244406   2.20618875]]

[-2.11975178  1.50069892 -0.36361168]


## Original featurization

This is the repliction of the featurization used in the original paper. It is saved as "data_with_features.csv"

It uses the csv that includes various targets, as well as using 388 .xyz files containing atomic coordinates in the silanols_vicinal folder

For *data_with_features.csv*
- Features: ['r1', 'r2', 'shaft_length', 'theta1', 'theta2', 'torsion']
- Labels: [**'rcat'**]

In [30]:
def r1r2(F1,F2,F3,F4):
    #F1-4 are xyz coords
    # I'm looking at these from a top down view i.e. OH groups facing out of the screen
    # F1 is the top left, it's the first F [-2.11975178  1.50069892 -0.36361168]
    # F2 is the next one. bottom left [-2.37533734 -1.23103682 -0.71987053]
    # F3 is the next, bottom right [ 2.33552808 -1.32539002 -0.55981726]
    # F4 is the next, top right [ 1.99841315  1.25144317 -0.89070369]
    r1 = np.linalg.norm(F1-F2)
    r2 = np.linalg.norm(F3-F4) #order doesn't matter here F1 or F2 first it is under sqrt

    #also find the midpoint of the two radii
    r1_mid = np.array([(F1[0]+F2[0])/2,(F1[1]+F2[1])/2,(F1[2]+F2[2])/2])
    r2_mid = np.array([(F3[0]+F4[0])/2,(F3[1]+F4[1])/2,(F3[2]+F4[2])/2])
    return r1,r2,r1_mid,r2_mid

In [31]:
def shaft(r1_mid,r2_mid):
    shaft_len = np.linalg.norm(r1_mid-r2_mid)
    return shaft_len

In [32]:
def thetas(F1,r1_mid,F4,r2_mid):
    u1 = F1-r1_mid # vector facing up r1 from midpoint
    v1 = r2_mid-r1_mid # vector facing to the right from r1midpoint to r2midpoint

    # we only want the theta in the xy plane so remove the z coord
    u1 = u1[0:2]
    v1 = v1[0:2]

    theta1 = np.arccos(np.dot(u1,v1)/(np.linalg.norm(u1)*np.linalg.norm(v1))) # formula for angle between vectors

    u2 = F4-r2_mid # vector facing up r2 from midpoint
    v2 = r1_mid-r2_mid # vector facing to the right from r2midpoint to r1midpoint

    # we only want the theta in the xy plane so remove the z coord
    u2 = u2[0:2]
    v2 = v2[0:2]

    theta2 = np.arccos(np.dot(u2,v2)/(np.linalg.norm(u2)*np.linalg.norm(v2))) # formula for angle between vectors

    return theta1,theta2

In [33]:
def torsion_funx(F4,r2_mid,F1,r1_mid):
    top = F4-r2_mid
    bottom = F1-r1_mid

    # removing coordinate similar to what was done for theta, now we only want the angle in the yz plane so remove x

    top = top[1:3]
    bottom = bottom[1:3]

    torsion = np.arccos(np.dot(top,bottom)/(np.linalg.norm(top)*np.linalg.norm(bottom)))
    return torsion

In [34]:
# finally make a function that spits out 6 lists and loops over every thing in the coord_list
def lists_out(coord_list):

    # Initialize lists
    rplus_list = []
    rminus_list = []
    shaft_list = []
    thetaplus_list = []
    thetaminus_list = []
    torsion_list = []

    for j in range(len(coord_list)):
        # define fluorine atom coords for j index
        F1,F2,F3,F4 = coord_list[j][1][0],coord_list[j][1][1],coord_list[j][1][2],coord_list[j][1][3]

        # find radii
        r1,r2,r1_mid,r2_mid = r1r2(F1,F2,F3,F4)

        # calculate r+ and r- as done in paper
        rplus = r1+r2
        rminus = r1-r2

        # find shaft length
        shaft_len = shaft(r1_mid,r2_mid)

        # find thetas
        theta1,theta2 = thetas(F1,r1_mid,F4,r2_mid)
        # convert units
        theta1 = np.rad2deg(theta1)
        theta2 = np.rad2deg(theta2)

        # calculate theta+ and theta-
        thetaplus = theta1+theta2
        thetaminus = theta1-theta2

        # find torsion
        torsion = torsion_funx(F4,r2_mid,F1,r1_mid)
        # convert units
        torsion = np.rad2deg(torsion)

        # now add all of these to the list at j spot
        rplus_list.append(rplus)
        rminus_list.append(rminus)
        shaft_list.append(shaft_len)
        thetaplus_list.append(thetaplus)
        thetaminus_list.append(thetaminus)
        torsion_list.append(torsion)
    # spit the lists out

    return rplus_list,rminus_list,shaft_list,thetaplus_list,thetaminus_list,torsion_list


In [35]:
rplus_list, rminus_list, shaft_list, thetaplus_list, thetaminus_list, torsion_list = lists_out(coord_list)

In [36]:
# make new df
outputdf = label_df.copy()

outputdf['r1'] = rplus_list
outputdf['r2'] = rminus_list
outputdf['shaft_length'] = shaft_list
outputdf['theta1'] = thetaplus_list
outputdf['theta2'] = thetaminus_list
outputdf['torsion'] = torsion_list

In [37]:
outputdf.head(400)

,index,name,k2,K1,rAC,rcat,kg(473K),kg(673K),kg(873K),pg(473K),...,pg(873K),reff(Tg=473K),reff(Tg=673K),reff(Tg=873K),r1,r2,shaft_length,theta1,theta2,torsion
0,0,coords_1Kps_001_0001,4.154248e+03,0.358224,1.127679e+03,1.127664e+03,1.799087e+01,63998.070000,5.905110e+06,1.000000e+00,...,1.0,1127.664000,1127.66400,1.127664e+03,5.386470,0.146928,4.421667,167.201469,6.565717,14.747488
1,1,coords_1Kps_002_0002,7.842454e+03,0.174889,1.207018e+03,1.207000e+03,6.089289e+01,162649.600000,1.291530e+07,1.000000e+00,...,1.0,1207.000000,1207.00000,1.207000e+03,5.480156,0.025942,4.482822,185.517513,6.196532,11.304056
2,2,coords_1Kps_002_0103,9.576523e+03,1.026440,4.944847e+03,4.944548e+03,5.152252e-06,1.538175,1.559147e+03,1.837715e-02,...,1.0,90.866690,4944.54800,4.944548e+03,5.411764,0.122305,3.959733,166.709309,-6.711944,1.613101
3,3,coords_1Kps_003_0003,5.906500e+04,0.553494,2.158001e+04,2.157430e+04,2.212001e+01,82010.950000,7.720031e+06,1.000000e+00,...,1.0,21574.300000,21574.30000,2.157430e+04,5.382705,-0.132749,4.302871,146.813691,0.724178,27.109136
4,4,coords_1Kps_003_0102,2.904114e+03,0.938815,1.434774e+03,1.434749e+03,5.070531e+01,138567.900000,1.108968e+07,1.000000e+00,...,1.0,1434.749000,1434.74900,1.434749e+03,5.110214,-0.099353,4.520485,164.223536,-0.614341,3.085213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,383,coords_1Kps_099_0102,1.630037e+05,0.716986,6.963183e+04,6.957246e+04,9.829145e-02,1782.579000,3.977919e+05,1.000000e+00,...,1.0,69572.460000,69572.46000,6.957246e+04,5.226315,0.040849,3.963426,149.838518,11.807033,9.469350
384,384,coords_1Kps_099_0406,7.200909e+04,1.001387,3.673744e+04,3.672090e+04,5.784885e+00,33583.320000,4.042024e+06,1.000000e+00,...,1.0,36720.900000,36720.90000,3.672090e+04,5.390237,0.027154,4.369727,165.445006,-1.752052,43.830134
385,385,coords_1Kps_099_0507,5.594144e+03,0.014340,8.221265e+01,8.221257e+01,1.550250e+01,61374.920000,5.960255e+06,1.000000e+00,...,1.0,82.212570,82.21257,8.221257e+01,5.183443,0.146010,4.252750,133.708083,-15.449385,36.586584
386,386,coords_1Kps_100_0103,2.651290e+08,0.201686,4.597372e+07,2.940699e+07,9.027036e-15,0.000001,2.864705e-02,3.249733e-11,...,1.0,0.000956,115987.60000,2.940699e+07,5.273317,0.104830,4.111392,208.624227,15.114690,34.892059


In [38]:
outputdf.to_csv(f'{data_dir}/data_with_features.csv', index=False)

## Custom featurization

This is a different featurization made as part of this project. It is saved as "data_with_features3.csv"

It uses the csv that includes various targets, as well as using 388 .xyz files containing atomic coordinates in the silanols_vicinal folder

For *data_with_features3.csv*
- Features:['Fa_theta_plus', 'Fa_theta_minus', 'Fa_torsion', 'Fa_len_plus', 'Fa_len_minus', 'Fb_theta_plus', 'Fb_theta_minus', 'Fb_torsion', 'Fb_len_plus', 'Fb_len_minus', 'O_theta_plus', 'O_theta_minus', 'O_torsion', 'O_len_plus', 'O_len_minus', 'center_torsion_plus', 'center_torsion_minus', 'center_height', 'center_r_plus', 'center_r_minus']
- Labels: [**'rcat'**]

In [51]:
# random ideas
# narrowing down regimes where certain features have a greater impact on reactivity
# maybe split up PCA graph into two sections and seeing if a more defnitive feature pops out, or if explained variance increases for both splits
# then you can sort of identify two regimes by which bonds are changing maybe
# maybe assuming symmetry between the two Si centers is an alright assumption
# having a probability 3D probability density of mass change with axes
# calculate position of atoms of interest if they are in fact definitively defined by the F coords

# The paper states that it at least qualitatively looks like the sticky outyness of the central O bond between the two Sis
# and how much the outward facing oxygen bonds point away from eachother (I think of this as how wide the W shape of the structure is)

# How can we make features to include at least one (ideally both) of these changes in the features
# this would allow for the principal components to have interpretable units as it will be a combination of these geometrically understood features
# Getting the two outward oxygen angles is somewhat easy

# could be just having all the angles of each Si tetrahedron
# Actually I reckon this wouldn't work too well, both less and more reactive sites seem to have a 90 deg angle, but the two 90 deg angles are more tilted relative to eachother

# an alternative to the above could be

# could also be the distance between every atom, with the two O2s included

# I am noticing now that the most reactive site has the central O between the two Sis pointed in the same direction as the hydrogens (relative to the Sis I guess)
# maybe this and other features could be captured by some metric that measures the similarity between two vector's direction

In [52]:
# ['F', 'F', 'F', 'F', 'Si', 'Si', 'O', 'O', 'O', 'H', 'H']
# first Si in the negative direction (left) is Si1
# first O is the center O
# 2nd and 3rd O is the left and right O attached to the Si respectively

# left is negative x and right is positive x, looking at the xz plane with y going into the screen

def W_angle(Si1,Si2,Out1,Out2):

    # inputs come in as lists so change to arrays so addition can be done
    Si1 = np.array(Si1)
    Si2 = np.array(Si2)
    Out1 = np.array(Out1)
    Out2 = np.array(Out2)

    u1 = Out1-Si1 # vector facing up to Sticking out atom from left Si
    v1 = Si2-Si1 # vector facing to the right from Si1 to Si2

    theta1 = np.arccos(np.dot(u1,v1)/(np.linalg.norm(u1)*np.linalg.norm(v1))) # formula for angle between vectors

    u2 = Out2-Si2 # vector facing up to Sticking out atom from right Si
    v2 = Si1-Si2 # vector facing to the right from Si1 to Si2

    theta2 = np.arccos(np.dot(u2,v2)/(np.linalg.norm(u2)*np.linalg.norm(v2))) # formula for angle between vectors

    theta_plus = theta1+theta2
    theta_minus = theta1 - theta2

    # both of these thetas should be taken with vectors projected onto the same plane like they were in the normal featurization.
    # but this is good enough

    # now get ratios of radii of sticking out atoms
    len1 = np.linalg.norm(u1) # left radii

    len2 = np.linalg.norm(u2) # right radii

    len_plus = len1 + len2
    len_minus = len1 - len2

    # don't necessarily need radii of distance between the Sis as this is specified in the CenterO features

    # now calculating torsion between the two sticking out atoms and the Sis

    top = Out2-Si2
    bottom = Out1-Si1

    # only consider the yz plane, eliminate the x

    top = top[1:3]
    bottom = bottom[1:3]

    torsion = np.arccos(np.dot(top,bottom)/(np.linalg.norm(top)*np.linalg.norm(bottom)))

    return theta_plus,theta_minus,torsion

# ['F', 'F', 'F', 'F', 'Si', 'Si', 'O', 'O', 'O', 'H', 'H']
# first Si in the negative direction (left) is Si1
# first O is the center O
# 2nd and 3rd O is the left and right O attached to the Si respectively

# left is negative x and right is positive x, looking at the xz plane with y going into the screen

def W_lens(Si1,Si2,Out1,Out2):

    # inputs come in as list so change to arrays

    Si1 = np.array(Si1)
    Si2 = np.array(Si2)
    Out1 = np.array(Out1)
    Out2 = np.array(Out2)

    r1 = Out1-Si1 # vector facing up to Sticking out atom from left Si
    r2 = Out2-Si2 # vector facing to the right from Si1 to Si2

    # calculate ratios

    len1 = np.linalg.norm(r1) # left radii

    len2 = np.linalg.norm(r2) # right radii

    len_plus = len1 + len2
    len_minus = len1 - len2

    return len_plus,len_minus

def all_W_angles(some_row):

    F1a,F1b,F2b,F2a,centerO,O1,O2,Si1,Si2,H1,H2 = some_row # unpack the input list

    # input is just every row, spits out list of:
    # theta plus minus and torsion, then plus and minus lengths, for each group of atoms that form a W
    # in the order Fas, Fbs, Os

    list_to_append = list(W_angle(Si1,Si2,F1a,F2a))
    list_to_append.extend(list(W_lens(Si1,Si2,F1a,F2a)))
    list_to_append.extend(list(W_angle(Si1,Si2,F1b,F2b)))
    list_to_append.extend(list(W_lens(Si1,Si2,F1b,F2b)))
    list_to_append.extend(list(W_angle(Si1,Si2,O1,O2)))
    list_to_append.extend(list(W_lens(Si1,Si2,O1,O2)))


    return list_to_append

In [53]:
def centerO_funx(some_row):

    F1a,F1b,F2b,F2a,centerO,O1,O2,Si1,Si2,H1,H2 = some_row # unpack the input list

    # inputs come in as list so change to arrays

    centerO = np.array(centerO)
    Si1 = np.array(Si1)
    Si2 = np.array(Si2)
    O1 = np.array(O1)
    O2 = np.array(O2)

    # recalculate outward facing vectors for O1 and O2
    OutO_1_vec = O1 - Si1
    OutO_2_vec = O2 - Si2

    l_vec_OG = Si1 - centerO # vector facing from center O to left Si
    r_vec_OG = Si2 - centerO # vector facing from center O to right Si

    # get lengths of bonds and calculate ratio
    l_len = np.linalg.norm(l_vec_OG)
    r_len = np.linalg.norm(r_vec_OG)

    center_r_plus = l_len + r_len
    center_r_minus = l_len - r_len

    # only want the components in the zx plane
    l_vec = np.array([l_vec_OG[0],l_vec_OG[2]])
    r_vec = np.array([r_vec_OG[0],r_vec_OG[2]])

    # now find torsion angles respective to each silicon out Oxygen
    # only in zy plane so get rid of x
    OutO_1_vec = OutO_1_vec[1:3]
    OutO_2_vec = OutO_2_vec[1:3]
    l_vec = l_vec_OG[1:3]
    r_vec = r_vec_OG[1:3]

    torsion1 = np.arccos(np.dot(OutO_1_vec,l_vec)/(np.linalg.norm(OutO_1_vec)*np.linalg.norm(l_vec)))

    torsion2 = np.arccos(np.dot(OutO_2_vec,r_vec)/(np.linalg.norm(OutO_2_vec)*np.linalg.norm(r_vec)))

    # had some issues with the term inside the arccos part being -1.00000002 and causing a nan due to being out of range of the arccos function
    # making this to enforce the proper value of the arcos of -1 being pi and not nan

    if np.isnan(torsion1) == True:
        torsion1 = np.pi
    if np.isnan(torsion2) == True:
        torsion2 = np.pi

    # use addition and difference of features like the original paper thetas
    torsion_plus = torsion1 + torsion2
    torsion_minus = torsion1 - torsion2

    # need vector between the two Sis only in zx plane
    Si_vec = Si2 - Si1
    Si_vec = np.array([Si_vec[0],Si_vec[2]])

    # only using the left angle fo this featurization because it shouldnt matter both sides will have the same information on the stickyoutyness of the center O
    center_phi = np.arccos(np.dot(l_vec,Si_vec)/(np.linalg.norm(l_vec)*np.linalg.norm(Si_vec)))

    # now calculate the h*cosine(phi) to get stickyoutness
    stickyouty = l_len*np.cos(center_phi)

    return torsion_plus,torsion_minus,stickyouty,center_r_plus,center_r_minus


In [54]:
# finally make a function that spits out feature dataframe and loops over every thing in the coord_list
def lists_out(coord_list):

    # Initialize dataframe

    feature_df = pd.DataFrame()

    # inialize CenterO funx lists

    torsion_plus_list = []
    torsion_minus_list = []
    stickyouty_list = []
    center_r_plus_list = []
    center_r_minus_list = []

    # initialize the W lists for each pair of atoms

    Fa_theta_plus_list = []
    Fa_theta_minus_list = []
    Fa_torsion_list = []
    Fa_len_plus_list = []
    Fa_len_minus_list = []

    Fb_theta_plus_list = []
    Fb_theta_minus_list = []
    Fb_torsion_list = []
    Fb_len_plus_list = []
    Fb_len_minus_list = []

    O_theta_plus_list = []
    O_theta_minus_list = []
    O_torsion_list = []
    O_len_plus_list = []
    O_len_minus_list = []

    for j in range(len(coord_list)):

        #flatten array at the row to a list of lists since this is what the functions I made take as an input
        current_row = coord_list[j][1].tolist()

        output_list = all_W_angles(current_row)
        # I wish I had done this in a less strange way but oh well
        Fa_theta_plus_list.append(output_list[0])
        Fa_theta_minus_list.append(output_list[1])
        Fa_torsion_list.append(output_list[2])
        Fa_len_plus_list.append(output_list[3])
        Fa_len_minus_list.append(output_list[4])

        Fb_theta_plus_list.append(output_list[5])
        Fb_theta_minus_list.append(output_list[6])
        Fb_torsion_list.append(output_list[7])
        Fb_len_plus_list.append(output_list[8])
        Fb_len_minus_list.append(output_list[9])

        O_theta_plus_list.append(output_list[10])
        O_theta_minus_list.append(output_list[11])
        O_torsion_list.append(output_list[12])
        O_len_plus_list.append(output_list[13])
        O_len_minus_list.append(output_list[14])

        # now just do this for center O and output and fix inevitable bugs

        center_O_output = centerO_funx(current_row)

        # put output into lists

        torsion_plus_list.append(center_O_output[0])
        torsion_minus_list.append(center_O_output[1])
        stickyouty_list.append(center_O_output[2])
        center_r_plus_list.append(center_O_output[3])
        center_r_minus_list.append(center_O_output[4])

    # turn lists into dataframe
    feature_df['Fa_theta_plus'] = Fa_theta_plus_list
    feature_df['Fa_theta_minus'] = Fa_theta_minus_list
    feature_df['Fa_torsion'] = Fa_torsion_list
    feature_df['Fa_len_plus'] = Fa_len_plus_list
    feature_df['Fa_len_minus'] = Fa_len_minus_list

    feature_df['Fb_theta_plus'] = Fb_theta_plus_list
    feature_df['Fb_theta_minus'] = Fb_theta_minus_list
    feature_df['Fb_torsion'] = Fb_torsion_list
    feature_df['Fb_len_plus'] = Fb_len_plus_list
    feature_df['Fb_len_minus'] = Fb_len_minus_list

    feature_df['O_theta_plus'] = O_theta_plus_list
    feature_df['O_theta_minus'] = O_theta_minus_list
    feature_df['O_torsion'] = O_torsion_list
    feature_df['O_len_plus'] = O_len_plus_list
    feature_df['O_len_minus'] = O_len_minus_list

    feature_df['center_torsion_plus'] = torsion_plus_list
    feature_df['center_torsion_minus'] = torsion_minus_list
    feature_df['center_height'] = stickyouty_list # this label is slightly confusing, despite being a theta this is not an angle but a length of how far the center oxygen sticks out between the line between the Si's
    feature_df['center_r_plus'] = center_r_plus_list
    feature_df['center_r_minus'] = center_r_minus_list

    return feature_df


In [55]:
feature_df = lists_out(coord_list)

/tmp/ipython-input-568794414.py:38: RuntimeWarning: invalid value encountered in arccos
  torsion1 = np.arccos(np.dot(OutO_1_vec,l_vec)/(np.linalg.norm(OutO_1_vec)*np.linalg.norm(l_vec)))


In [56]:
feature_df.head(100)

,Fa_theta_plus,Fa_theta_minus,Fa_torsion,Fa_len_plus,Fa_len_minus,Fb_theta_plus,Fb_theta_minus,Fb_torsion,Fb_len_plus,Fb_len_minus,O_theta_plus,O_theta_minus,O_torsion,O_len_plus,O_len_minus,center_torsion_plus,center_torsion_minus,center_height,center_r_plus,center_r_minus
0,3.293172,-0.077100,0.311678,5.483570,-0.102064,3.523059,0.125022,0.198111,5.455464,-0.103517,1.284863,-0.358128,0.322883,6.528657,1.092253,6.208192,0.074993,-0.211745,5.510534,-2.070577
1,3.842869,0.363175,0.232807,5.402661,-0.065523,3.764248,-0.216311,0.315246,5.157857,-0.007258,1.541221,-0.504524,0.303247,6.136772,0.464089,6.270120,0.013066,0.207564,5.049563,-1.629590
2,2.832844,-0.138224,0.105074,5.386619,0.045142,3.071761,0.067012,0.037984,5.489241,-0.132949,1.004226,-0.209584,0.271315,6.843151,1.462100,6.137602,0.145583,-0.080094,5.796059,-2.415823
3,2.965641,0.432033,0.500611,5.438158,0.024400,3.524934,-0.116356,0.468631,5.375639,0.139081,1.212959,-0.299621,0.254058,6.432922,1.037949,6.086755,0.196430,0.345823,5.705212,-2.224856
4,3.154948,0.054604,0.099346,5.320945,-0.057706,3.412771,-0.037349,0.067255,5.461579,0.024318,1.098932,-0.270457,0.043525,6.704263,1.366985,6.240064,0.043121,0.066389,5.704681,-2.290114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.507833,0.085072,0.264309,5.224980,-0.117034,3.388656,-0.202534,0.303434,5.575411,0.052836,1.276854,-0.351413,0.334863,6.442748,1.274158,6.279219,0.003967,0.302757,5.490886,-2.020646
96,2.927415,0.011366,0.442854,5.360398,0.017602,2.876519,-0.146753,0.499415,5.394544,-0.009191,0.869811,-0.108592,0.609906,6.725365,1.704219,6.217265,0.065920,0.516907,5.748546,-2.338914
97,3.791569,0.268760,0.392687,5.341990,-0.085460,3.478868,-0.347763,0.385577,5.539685,0.014850,1.381580,-0.400601,0.332102,6.238746,0.999429,6.271215,0.011971,0.274853,5.277316,-1.877913
98,2.694160,-0.004829,0.104967,5.469942,0.199545,2.754835,0.046003,0.273564,5.401540,0.076497,0.744719,-0.079913,0.391534,7.097773,1.564004,6.190699,0.092487,-0.271230,6.055323,-2.624474


In [57]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Fa_theta_plus         388 non-null    float64
 1   Fa_theta_minus        388 non-null    float64
 2   Fa_torsion            388 non-null    float64
 3   Fa_len_plus           388 non-null    float64
 4   Fa_len_minus          388 non-null    float64
 5   Fb_theta_plus         388 non-null    float64
 6   Fb_theta_minus        388 non-null    float64
 7   Fb_torsion            388 non-null    float64
 8   Fb_len_plus           388 non-null    float64
 9   Fb_len_minus          388 non-null    float64
 10  O_theta_plus          388 non-null    float64
 11  O_theta_minus         388 non-null    float64
 12  O_torsion             388 non-null    float64
 13  O_len_plus            388 non-null    float64
 14  O_len_minus           388 non-null    float64
 15  center_torsion_plus   3

In [58]:
data_with_features3 = pd.concat([label_df, feature_df], axis=1)

In [59]:
pd.set_option('display.max_columns', None)
data_with_features3.head()

,index,name,k2,K1,rAC,rcat,kg(473K),kg(673K),kg(873K),pg(473K),pg(673K),pg(873K),reff(Tg=473K),reff(Tg=673K),reff(Tg=873K),Fa_theta_plus,Fa_theta_minus,Fa_torsion,Fa_len_plus,Fa_len_minus,Fb_theta_plus,Fb_theta_minus,Fb_torsion,Fb_len_plus,Fb_len_minus,O_theta_plus,O_theta_minus,O_torsion,O_len_plus,O_len_minus,center_torsion_plus,center_torsion_minus,center_height,center_r_plus,center_r_minus
0,0,coords_1Kps_001_0001,4154.248,0.358224,1127.679,1127.664,17.990870,63998.070000,5.905110e+06,1.000000,1.0,1.0,1127.66400,1127.664,1127.664,3.293172,-0.077100,0.311678,5.483570,-0.102064,3.523059,0.125022,0.198111,5.455464,-0.103517,1.284863,-0.358128,0.322883,6.528657,1.092253,6.208192,0.074993,-0.211745,5.510534,-2.070577
1,1,coords_1Kps_002_0002,7842.454,0.174889,1207.018,1207.000,60.892890,162649.600000,1.291530e+07,1.000000,1.0,1.0,1207.00000,1207.000,1207.000,3.842869,0.363175,0.232807,5.402661,-0.065523,3.764248,-0.216311,0.315246,5.157857,-0.007258,1.541221,-0.504524,0.303247,6.136772,0.464089,6.270120,0.013066,0.207564,5.049563,-1.629590
2,2,coords_1Kps_002_0103,9576.523,1.026440,4944.847,4944.548,0.000005,1.538175,1.559147e+03,0.018377,1.0,1.0,90.86669,4944.548,4944.548,2.832844,-0.138224,0.105074,5.386619,0.045142,3.071761,0.067012,0.037984,5.489241,-0.132949,1.004226,-0.209584,0.271315,6.843151,1.462100,6.137602,0.145583,-0.080094,5.796059,-2.415823
3,3,coords_1Kps_003_0003,59065.000,0.553494,21580.010,21574.300,22.120010,82010.950000,7.720031e+06,1.000000,1.0,1.0,21574.30000,21574.300,21574.300,2.965641,0.432033,0.500611,5.438158,0.024400,3.524934,-0.116356,0.468631,5.375639,0.139081,1.212959,-0.299621,0.254058,6.432922,1.037949,6.086755,0.196430,0.345823,5.705212,-2.224856
4,4,coords_1Kps_003_0102,2904.114,0.938815,1434.774,1434.749,50.705310,138567.900000,1.108968e+07,1.000000,1.0,1.0,1434.74900,1434.749,1434.749,3.154948,0.054604,0.099346,5.320945,-0.057706,3.412771,-0.037349,0.067255,5.461579,0.024318,1.098932,-0.270457,0.043525,6.704263,1.366985,6.240064,0.043121,0.066389,5.704681,-2.290114


In [60]:
data_with_features3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 388 non-null    int64  
 1   name                  388 non-null    object 
 2   k2                    388 non-null    float64
 3   K1                    388 non-null    float64
 4   rAC                   388 non-null    float64
 5   rcat                  388 non-null    float64
 6   kg(473K)              388 non-null    float64
 7   kg(673K)              388 non-null    float64
 8   kg(873K)              388 non-null    float64
 9   pg(473K)              388 non-null    float64
 10  pg(673K)              388 non-null    float64
 11  pg(873K)              388 non-null    float64
 12  reff(Tg=473K)         388 non-null    float64
 13  reff(Tg=673K)         388 non-null    float64
 14  reff(Tg=873K)         388 non-null    float64
 15  Fa_theta_plus         3

In [61]:
data_with_features3.to_csv(f'{data_dir}/data_with_features3.csv', index=False)